In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from tqdm import tqdm
import numpy as np
import sklearn.metrics as metrics

## Hyperparameters and Model Architecture

In [12]:
HIDDEN_DIM = 64
DROPOUT_RATE = 0.2
LEARNING_RATE = 0.001
WEIGHT_DECAY = 0.0
BATCH_SIZE = 10
CRITERION = F.nll_loss
EPOCHS = 3

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [30]:
class DNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(DNN, self).__init__()
        self.input_dim = input_dim
        self.fc1 = nn.Linear(input_dim, HIDDEN_DIM)
        self.fc2 = nn.Linear(HIDDEN_DIM, HIDDEN_DIM)
        self.fc3 = nn.Linear(HIDDEN_DIM, HIDDEN_DIM)
        self.fc4 = nn.Linear(HIDDEN_DIM, HIDDEN_DIM)
        self.fc5 = nn.Linear(HIDDEN_DIM, HIDDEN_DIM)
        self.fc6 = nn.Linear(HIDDEN_DIM, output_dim)
        self.dropout = nn.Dropout(DROPOUT_RATE)
        self.grayscale = transforms.Grayscale()
        
    def forward(self, x):
        if x.shape[1] == 3:
            x = self.grayscale(x)
        x = x.view(-1, self.input_dim)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        x = self.dropout(F.relu(self.fc4(x)))
        x = self.dropout(F.relu(self.fc5(x)))
        x = F.log_softmax(self.fc6(x), dim=1)
        return x
        

### MNIST

In [28]:
train = datasets.MNIST("./347data", train=True, download=True, transform=transforms.Compose([transforms.ToTensor()]))
test = datasets.MNIST("./347data", train=False, download=True, transform=transforms.Compose([transforms.ToTensor()]))
validation_set_size = int(len(train) * 0.1)
training_set_size = len(train) - validation_set_size
train_set, validation_set = torch.utils.data.random_split(train, [training_set_size, validation_set_size])
train_set = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
validation_set = torch.utils.data.DataLoader(validation_set, batch_size=BATCH_SIZE, shuffle=True)
test_set = torch.utils.data.DataLoader(validation_set, batch_size=BATCH_SIZE, shuffle=True)

In [31]:
MNIST_dnn = DNN(28 * 28, 10).to(device)
optimizer = optim.Adam(MNIST_dnn.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
for epoch in range(EPOCHS):
    for data in tqdm(train_set):
        X, y = data
        MNIST_dnn.zero_grad()
        output = MNIST_dnn(X.to(device))
        loss = CRITERION(output, y.to(device))
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch + 1} Loss: {loss.item()}")

  1%|          | 30/5400 [00:00<00:18, 294.77it/s]

Epoch 1 Loss: 0.20145754516124725


  1%|          | 29/5400 [00:00<00:18, 287.71it/s]

Epoch 2 Loss: 0.0837847962975502


100%|██████████| 5400/5400 [00:21<00:00, 251.89it/s]

Epoch 3 Loss: 0.5969727635383606


In [32]:
output = []
true = []
MNIST_dnn.eval()
with torch.no_grad():
    for data in validation_set:
        X, y = data
        for i in MNIST_dnn(X.to(device)):
            output.append(torch.argmax(i).cpu())
        for i in y:
            true.append(i)
MNIST_dnn.train()
print("Validation Accuracy:", metrics.accuracy_score(true, output))
print("Validation F1 Score:", metrics.f1_score(true, output, average="macro"))
true = np.eye(10)[true]
output = np.eye(10)[output]
print("Validation AUC Score:", metrics.roc_auc_score(true, output, multi_class="ovo", average="macro"))
            

Validation Accuracy: 0.96
Validation F1 Score: 0.9598386143410804
Validation AUC Score: 0.9776275372677483


### CIFAR-10

In [33]:
train = datasets.CIFAR10("./347data", train=True, download=True, transform=transforms.Compose([transforms.ToTensor()]))
test = datasets.CIFAR10("./347data", train=False, download=True, transform=transforms.Compose([transforms.ToTensor()]))
validation_set_size = int(len(train) * 0.1)
training_set_size = len(train) - validation_set_size
train_set, validation_set = torch.utils.data.random_split(train, [training_set_size, validation_set_size])
train_set = torch.utils.data.DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
validation_set = torch.utils.data.DataLoader(validation_set, batch_size=BATCH_SIZE, shuffle=True)
test_set = torch.utils.data.DataLoader(test, batch_size=BATCH_SIZE, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [35]:
CIFAR10_dnn = DNN(32 * 32, 10).to(device)
optimizer = optim.Adam(CIFAR10_dnn.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
for epoch in range(EPOCHS):
    for data in tqdm(train_set):
        X, y = data
        CIFAR10_dnn.zero_grad()
        output = CIFAR10_dnn(X.to(device))
        loss = CRITERION(output, y.to(device))
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch + 1} Loss: {loss.item()}")

  0%|          | 22/5000 [00:00<00:22, 217.09it/s]

Epoch 1 Loss: 2.153488874435425


  0%|          | 20/5000 [00:00<00:25, 193.51it/s]

Epoch 2 Loss: 1.8779704570770264


100%|██████████| 5000/5000 [00:22<00:00, 226.63it/s]

Epoch 3 Loss: 2.2098474502563477


In [36]:
output = []
true = []
CIFAR10_dnn.eval()
with torch.no_grad():
    for data in validation_set:
        X, y = data
        for i in CIFAR10_dnn(X.to(device)):
            output.append(torch.argmax(i).cpu())
        for i in y:
            true.append(i)
CIFAR10_dnn.train()
print("Validation Accuracy:", metrics.accuracy_score(true, output))
print("Validation F1 Score:", metrics.f1_score(true, output, average="macro"))
true = np.eye(10)[true]
output = np.eye(10)[output]
print("Validation AUC Score:", metrics.roc_auc_score(true, output, multi_class="ovo", average="macro"))
            

Validation Accuracy: 0.2254
Validation F1 Score: 0.20074097433389385
Validation AUC Score: 0.5690189593265054


### Iyer

In [37]:
iyer = np.loadtxt(open("347data/iyer.txt", "rb"), delimiter="\t")
features = iyer[:, 2:].astype(float)
labels = iyer[:, 1].astype(int)

data = [] #data0
for i in range(features.shape[0]):
    stack = np.array([])
    stack = np.column_stack([np.roll(features[i,], j, axis=0) for j in range(features.shape[1])]).astype(np.float32)
    data.append([stack, np.array(labels[i] + 1, dtype=int)])

data = np.array(data)
np.random.shuffle(data)
X = torch.tensor(np.array([i[0] for i in data])).view(-1, 1, 12, 12)
y = torch.tensor(np.array([i[1] for i in data]))

test_set_size = int(X.shape[0] * 0.1)
training_set_size = X.shape[0] - test_set_size
validation_set_size = int(training_set_size * 0.1)
training_set_size -= validation_set_size
print(training_set_size, validation_set_size, test_set_size)
train_X = X[:training_set_size]
train_y = y[:training_set_size]
validation_X = X[training_set_size:training_set_size + validation_set_size]
validation_y = y[training_set_size:training_set_size + validation_set_size]
test_X = X[training_set_size + validation_set_size:]
test_y = y[training_set_size + validation_set_size:]

420 46 51


In [39]:
Iyer_dnn = DNN(12 * 12, 12).to(device)
optimizer = optim.Adam(Iyer_dnn.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
for epoch in range(EPOCHS):
    for i in tqdm(range(0, training_set_size, BATCH_SIZE)):
        batch_X = train_X[i:i+BATCH_SIZE]
        batch_y = train_y[i:i + BATCH_SIZE]
        
        Iyer_dnn.zero_grad()
        output = Iyer_dnn(batch_X.to(device))
        loss = CRITERION(output, batch_y.to(device))
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch + 1} Loss: {loss.item()}")
        

100%|██████████| 42/42 [00:00<00:00, 491.79it/s]

Epoch 1 Loss: 2.1298329830169678
Epoch 2 Loss: 1.6145617961883545
Epoch 3 Loss: 1.3383407592773438


In [41]:
output = []
true = []
Iyer_dnn.eval()
with torch.no_grad():
    for i in range(validation_set_size):
        output.append(torch.argmax(Iyer_dnn(validation_X[i].view(-1, 1, 12, 12).to(device))).cpu())
        true.append(validation_y[i])
Iyer_dnn.train()
print("Validation Accuracy:", metrics.accuracy_score(true, output))
print("Validation F1 Score:", metrics.f1_score(true, output, average="macro"))
labels = [0 for _ in range(0, 12)]
for i in true:
    labels[i] += 1
for i, l in enumerate(labels):
    if l == 0:
        true.append(i) 
        output.append(i)
labels = [0 for _ in range(0, 12)]
for i in output:
    labels[i] += 1
for i, l in enumerate(labels):
    if l == 0:
        true.append(i) 
        output.append(i)
true = np.eye(12)[true]
output = np.eye(12)[output]
print("Validation AUC Score:", metrics.roc_auc_score(true, output, multi_class="ovo", average="macro"))

Validation Accuracy: 0.41304347826086957
Validation F1 Score: 0.11273532668881506
Validation AUC Score: 0.7080428196572216
